# Experiment 5

- Adversarial input to emphasize benefit of our algorithm, even online

Schedulers:
```
schedulers = [
        MRIS(sort='WSVF'),
        ListScheduler(sort='WSVF', online=True),
        OnlineListScheduler(sort='WSVF'),
        TetrisScheduler(score_averaging='instantaneous'),
        BFEXECScheduler()
]
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scienceplots
import itertools
import matplotlib

In [ ]:
plt.style.use(['science'])
params = {
    "font.family": "serif",
    "text.usetex": True,
    'text.latex.preamble':
        r"""
        \usepackage{libertine}
        \usepackage[libertine]{newtxmath}
        """,
}
# https://matplotlib.org/stable/gallery/lines_bars_and_markers/linestyles.html
def get_linestyles():
     return itertools.cycle([
         ('solid', (0, ())),
         # ('loosely dotted',        (0, (1, 10))),
         ('dotted',                (0, (1, 1))),
         # ('densely dotted',        (0, (1, 1))),
         ('long dash with offset', (5, (10, 3))),
         ('loosely dashed',        (0, (5, 10))),
         #('dashed',                (0, (5, 5))),
         ('densely dashed',        (0, (5, 1))),
    
         #('loosely dashdotted',    (0, (3, 10, 1, 10))),
         ('dashdotted',            (0, (3, 5, 1, 5))),
         ('densely dashdotted',    (0, (3, 1, 1, 1))),
    
         ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
         ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
         ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))
    ])
mpl.rcParams.update(params)

In [ ]:
df = pd.read_parquet('adversarial.parquet')

In [ ]:
schedulers = df['scheduler'].unique().tolist()
schedulers.remove('List-WSVF')

In [ ]:
schedulers

# Plot the profile of each scheduler

In [ ]:
R = 3
M = 1
resource_idx = 0
N = 2500
demand_levels = 10
fig, axs = plt.subplots(nrows=2, ncols=2, tight_layout=True, dpi=200)
fig.set_figheight(3)
fig.set_figwidth(6)


# axs return value depends on the input rows and cols. Convert to always use 2D ndarray
if type(axs) is not np.ndarray:
    axs = np.array([[axs]])
elif axs.ndim == 1:
    axs = axs.reshape(1, -1)

axes = axs.flat
    
for s, scheduler in enumerate(schedulers):
    if scheduler == 'Tetris-instantaneous':
        label = r'\textsc{Tetris}'
    else:
        label = scheduler
        
    df_ = df[df['scheduler'] == scheduler]
    start_times = df_['S'].to_numpy()
    completion_times = df_['C'].to_numpy()
    demands = df_['d'].to_numpy()
    
    x = np.concatenate([np.zeros(1), start_times, completion_times, start_times-1E-8, completion_times-1E-8])
    x = np.unique(x)
    
    x = np.sort(x)
    cumulative_resource = np.zeros_like(x)
    
    stacked_data = []
    
    for i in range(N):
        cumulative_resource += np.where(((start_times[i]) <= x) & (completion_times[i] > x), demands[i][resource_idx], 0) / demand_levels
        stacked_data.append(np.copy(cumulative_resource))
    stacked_data.reverse()
    for j, data in enumerate(stacked_data):
        axes[s].stackplot(x, data, rasterized=True)
        axes[s].set_ylim(0, 1.1)
        axes[s].set_title(label)
        
fig.text(0.5, -0.00, 'Time [arb. units]', ha='center')
fig.text(-0.01, 0.5, 'Resource usage [arb. units]', va='center', rotation='vertical')
plt.savefig('adversarial.pdf', bbox_inches='tight')
plt.show()